# Review: Reinforcement Learning from Human Feedback

RLHF basically (and I say basically rather cheekily) this process:

### 1) 🏁 Start with a Model

- This could be a base model, or a model that's gone through some supervised fine-tuning. For example: instruction-tuning, chat-tuning, summarization-tuning, etc.

- Give the model prompts and have it generate multiple completions.

### 2) 🗣️ Collect Human Feedback

Ask humans to evaluate these responses, selecting the preferred ones, or ranking them according to certain criteria.

### 3) 💳 Train a Reward Model

- Use the collected human feedback to train a reward model.

- This model learns to predict the human preference between pairs of responses.

- The reward model approximates human judgments, assigning higher scores to more preferable responses.

🚨 **Important**: The reward model must output a scalar value score (ie, just a number) to each output generated by the LLM. This score is used for reinforcement learning, because it gives measures the output's alignment with human preferences.

### 4) 🤖 Do Reinforcement Learning

<img src="https://user-images.githubusercontent.com/17574157/213037462-5dd556de-3afa-4842-b546-0fc90e799249.png">


- Make a duplicate copy of the language model and freeze it's weights. Now you have two models: one whose weights will be updated via back prop, and one that will not be updated. The frozen LM acts as a quality control, so the trainable LM doesn't try to cheat the system.

- Generate completions from both the models. Measure how much the trainable LM's text predictions drift from the frozen one using KL divergence loss.

- Add that KL divergence loss to the reward from the reward model to keep things on track. Use the trained reward model to fine-tune the base model further.

- This step involves reinforcement learning algorithms, usually Proximal Policy Optimization (PPO).

- Optimize the policy of the base model to generate responses that maximize the predicted reward from the reward model.


## PPO Overview

This involves three steps which, to quote Hugging Face, is as follows: Rollout, Evaluate, and Optimize.

<img src="https://camo.githubusercontent.com/14c39f51b64e2fc3f1ed9da4af3f9027034544551aed3045d7a5930d603e233a/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f74726c2d696e7465726e616c2d74657374696e672f6578616d706c652d696d616765732f7265736f6c76652f6d61696e2f696d616765732f74726c5f6f766572766965772e706e67">

### 🧻 **Rollout**

The language model generates completions for some prompt or query.  Basically, generate tokens that can complete the sequence.

You then compute the reward for the completions and evaluate.

### 🔬 **Evaluate**

The query and response can be evaluated using a function, model, human feedback, or a combination.

This process should produce a single numerical value for each query/response pair.

You can check the scores of those tokens using the Reward Model.

### 💪🏽 **Optimize**

Using a trained model and a reference model (usually the pre-trained model before fine-tuning) calculate the log probabilities of the tokens in the sequences.

The KL divergence between the two outputs is then used as an additional reward signal to ensure that the generated responses do not deviate too far from the reference language model.

The active language model is trained with PPO and updated based on the scores and the generations of the reference model, which is the original model before RLHF.

# 🎯 DPO Overview

The core insight is to derive a loss function that directly solves the policy optimization and reward modeling steps simultaneously.

<img src="https://pbs.twimg.com/media/GBGxPDTa0AEyjUd?format=jpg">


## Let's zoom in on the loss function.

<img src="https://private-user-images.githubusercontent.com/1131538/301539636-1c4683af-7c65-4207-bd20-e2fc580a9542.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3MDY5MDIxMzksIm5iZiI6MTcwNjkwMTgzOSwicGF0aCI6Ii8xMTMxNTM4LzMwMTUzOTYzNi0xYzQ2ODNhZi03YzY1LTQyMDctYmQyMC1lMmZjNTgwYTk1NDIucG5nP1gtQW16LUFsZ29yaXRobT1BV1M0LUhNQUMtU0hBMjU2JlgtQW16LUNyZWRlbnRpYWw9QUtJQVZDT0RZTFNBNTNQUUs0WkElMkYyMDI0MDIwMiUyRnVzLWVhc3QtMSUyRnMzJTJGYXdzNF9yZXF1ZXN0JlgtQW16LURhdGU9MjAyNDAyMDJUMTkyMzU5WiZYLUFtei1FeHBpcmVzPTMwMCZYLUFtei1TaWduYXR1cmU9NTlkMmI5OTQzN2U2NzA4ZjQ1ZjQ5MzQ2NjQzZjJkYTQ5YjYwNjI1OTM0NjZmMDZlMDdkZTAxNmM4YThmYzcxZSZYLUFtei1TaWduZWRIZWFkZXJzPWhvc3QmYWN0b3JfaWQ9MCZrZXlfaWQ9MCZyZXBvX2lkPTAifQ._b_t5lTbTsCDI_HjJUdfJUq04WgL9vbY3UNNsgVJYm0">


1) Loss function of policy model with respect to reference model.

2) Expectation of the dataset with `x` samples of $y_{w}$ (chosen) and $y_{l}$ rejected outputs.

3) The logarithmic of the sigmoid applied to the argument.
  - In torch: `F.logsigmoid`
  - This will scale the result between 0 and 1, providing a probabilistic interpretation.

4) $\beta$: Hyperparameter that weights the importance of the deviation between the policy model and the reference model.

5) Log Probability ratio of the policy model’s probability of choosing the same  $y_{w}$  given the input `x`, divided by the reference model’s probability of choosing the same $y_{w}$  given the same input `x`.
 - These ratios indicate how much more or less likely the policy model is to choose a particular action compared to the reference model.


> Let's look at [Juan's implementation](https://github.com/jcolano/DPO/blob/main/dpo_loss.py) of the loss function for some intuition.

### DPO Pipieline

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*AqKOT0pxzi5kOgiobb-Fvg.png">

- Start, again, with two copies of your model. One with frozen weights and one with trainable weights

- Every sample gets a score from both the trainee LM and its frozen twin, based on how likely they are to spit out the expected response.

- Scores for chosen versus rejected responses to a given prompt are calculated using a simple binary classification approach.

- The model is trained to learn "good" and "bad" responses based on human preference data.

-  The chosen/rejected score is the product of probabilities assigned to each token in the chosen response sequence. This score reflects the model's assessment of the response's quality, with higher scores indicating a response more aligned with human preferences.

- The multiplication of probabilities across the sequence yields the final score for the response, which is used to guide the optimization of the model.

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*-l_uEgU-Qq-32kw4DgDgxg.png">


- The loss on the sample in DPO is calculated using the standard cross-entropy loss function. The model produces a probability distribution over the possible responses, and the cross-entropy loss measures the difference between the predicted probability distribution and the actual distribution (the ground truth).

- In the context of DPO, the actual distribution is based on human preferences, with the chosen response being the positive class and the rejected response being the negative class.

- The loss is minimized when the model's predictions align closely with the human-provided labels, effectively training the model to predict human preferences with higher accuracy.

##References:

- [Juan Olano's GitHub](https://github.com/jcolano/DPO)

- [João Lages blog post](https://medium.com/@joaolages/direct-preference-optimization-dpo-622fc1f18707)

# DPO vs RLHF

> [DPO is closer to RLHF than RLHF is to RL](https://youtu.be/3WU6fl7DHj0?t=4374)
>
> -- Nathan Lambert


<img src="https://i.ibb.co/mvFwnWc/dpo.gif">


DPO takes a simpler path to aligning LLMs than RLHF.

The RLHF pipeline involves multiple stages, including training a reward model on human feedback, using this model to generate rewards for policy training, and then using reinforcement learning algorithms to adjust the language model.

This is computationally intensive and requires careful hyperparameter tuning.

The DPO pipeline directly applies a classification loss to the language model, using human preference data to guide the optimization.

This sidesteps the intricate dance of RL algorithms and the meticulous adjustment of hyperparameters. As a result, DPO runs smoother and demands less computational power than RLHF, which often requires model sampling during its fine-tuning phase and can stumble into instability and higher computational needs.

In essence, DPO offers a streamlined and efficient alternative to the more *cumbersome* RLHF approach.


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [ ]:
import torch
import os
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import DPOTrainer

The `distilabel-intel-orca-dpo-pairs dataset` is a  refinement of the [Intel/orca_dpo_pairs](https://huggingface.co/datasets/Intel/orca_dpo_pairs) through [distilabel](https://github.com/argilla-io/distilabel), aligning more closely with AI community needs and model refinement practices.

Learn more about the dataset [here](https://huggingface.co/datasets/argilla/distilabel-intel-orca-dpo-pairs)

In [ ]:
dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/12859 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.filter(
    lambda r:
        r["status"] != "tie" and
        r["chosen_score"] >= 8 and
        not r["in_gsm8k_train"]
)

Filter:   0%|          | 0/12859 [00:00<?, ? examples/s]

# You need three features in your dataset

 - `prompt` -  the context prompt which is given to a model at inference time for text generation

 - `chosen` -  the preferred generated response to the prompt

 - `rejected` - the dispreferred response to the prompt

In [ ]:
def insert_prompts(example):
    """
    Inserts data into system_prompt and user_prompt placeholders.

    Args:
        example (dict): A dictionary representing a record in the dataset.

    Returns:
        dict: The modified record with the completed prompt.
    """
    system_prompt_template = "### System:\n{system_prompt}\n### User:\n{user_prompt}\n### Assistant:"
    system_prompt = example["system"]
    user_prompt = example["input"]


    filled_prompt = system_prompt_template.format(system_prompt=system_prompt, user_prompt=user_prompt)

    example["prompt"] = filled_prompt
    return example

dataset = dataset.map(insert_prompts)

Map:   0%|          | 0/5922 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.shuffle(seed=42).train_test_split(seed=42, test_size=.3)

In [ ]:
dataset = dataset.remove_columns(['generations', 'order', 'labelling_model', 'labelling_prompt', 'raw_labelling_response', 'rating', 'rationale', 'status', 'original_chosen', 'original_rejected', 'chosen_score', 'in_gsm8k_train'])

In [ ]:
train_data = dataset["train"]

test_data = dataset["test"]

In [ ]:
model_name = "Deci/DeciLM-7B-instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.pad_token_id =  tokenizer.eos_token_id

# Instantiate policy model

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

policy_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

policy_model = prepare_model_for_kbit_training(policy_model)

policy_model.config.pad_token_id = tokenizer.pad_token_id

policy_model.config.use_cache = False

In [ ]:
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["gate_proj", "down_proj", "up_proj"]
)

# Instantiate reference model

In [ ]:
reference_model_name = "Deci/DeciLM-7B-instruct"

bnb_config_ref = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
)

reference_model = AutoModelForCausalLM.from_pretrained(
    reference_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code = True,
    low_cpu_mem_usage=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Set training arguments

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_32bit",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_steps=10,
        logging_steps=10,
        learning_rate=5e-7,
        eval_steps=20,
        #num_train_epochs=1,
        max_steps=100,
        warmup_steps=20,
        lr_scheduler_type="cosine",
        remove_unused_columns=True
)

# Train!

In [ ]:
trainer = DPOTrainer(
    policy_model,
    reference_model,
    max_length=8196,
    max_prompt_length=4096,
    args=training_arguments,
    beta=0.1,
    peft_config=peft_config,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:314: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/4145 [00:00<?, ? examples/s]

Map:   0%|          | 0/1777 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 2
***** Running training *****
  Num examples = 4,145
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 100
  Number of trainable parameters = 28,311,552
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
20,0.693200,0.686885,0.001205,-0.011555,0.713161,0.012760,-200.757095,-173.131744,-3.033190,-2.918777


Saving model checkpoint to ./results/tmp-checkpoint-10
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Deci--DeciLM-7B-instruct/snapshots/abb8ae6b66ddf5986fe899d52d62dc17c54aa376/config.json
Model config DeciLMConfig {
  "architectures": [
    "DeciLMForCausalLM"
  ],
  "attention_bias": false,
  "auto_map": {
    "AutoConfig": "Deci/DeciLM-7B-instruct--configuration_decilm.DeciLMConfig",
    "AutoModelForCausalLM": "Deci/DeciLM-7B-instruct--modeling_decilm.DeciLMForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "deci",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "num_key_value_heads_per_layer": [
    4,
    4,
    4,
    4,
    4,
    2,
    2,
    2,
    2,
    2,
    4,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
   